In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [3]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2014','era_2014.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2014-01-01 ... 2014-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:33 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2014-01-01 ... 2014-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:33 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                     
2014-01-01 00:00:00 1000.0         49.25    -124.85    99.058105  0.005541
                                            -124.60    98.052246  0.005465
                                            -124.35    98.101074  0.005360
                                            -124.10    97.770996  0.005306
                                            -123.85    95.632324  0.005345
...                                                          ...       ...
2014-12-31 20:00:00 1000.0         24.50    -68.10     72.076538  0.013203
                                            -67.85     72.193726  0.013181
                                            -67.60     73.537476  0.013339
                                            -67.35     75.508179  0.013603
                                            -67.10     77.529663  0.013905

[67744000 rows x 2 columns]

In [6]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2014-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:33 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2014-12-31 1000.0         49.25    -124.85    82.009308  0.006447
                                   -124.60    79.446182  0.006442
                                   -124.35    77.228058  0.006349
                                   -124.10    74.945915  0.006267
                                   -123.85    72.010170  0.006265
...                                                 ...       ...
                          24.50    -68.10     77.920502  0.014710
                                   -67.85     77.977684  0.014703
                                   -67.60     78.029129  0.014694
                                   -67.35     78.068413  0.014682
                                   -67.10     78.113243  0.014669

[23200 rows x 2 columns]

In [8]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2014-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 82.01 ... ...
    q               (valid_time, latitude, longitude) float32 93kB 0.006447 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:33 GRIB to CDM+CF via cfgrib-0.9.1...

In [9]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2014_county_shapefile','cb_2014_us_county_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,01,001,00161526,0500000US01001,01001,Autauga,06,1539584444,25773561,"POLYGON Z ((-86.92120 32.65754 0.00000, -86.92..."
1,01,005,00161528,0500000US01005,01005,Barbour,06,2291820953,50864677,"POLYGON Z ((-85.74803 31.61918 0.00000, -85.74..."
2,01,023,00161537,0500000US01023,01023,Choctaw,06,2365954803,19059247,"POLYGON Z ((-88.47323 31.89386 0.00000, -88.46..."
3,01,033,00161542,0500000US01033,01033,Colbert,06,1534878355,80029923,"POLYGON Z ((-88.13925 34.58779 0.00000, -88.13..."
4,01,047,00161549,0500000US01047,01047,Dallas,06,2534807336,39134779,"POLYGON Z ((-87.47308 32.30761 0.00000, -87.42..."
...,...,...,...,...,...,...,...,...,...,...
3228,17,015,00424209,0500000US17015,17015,Carroll,06,1152707810,55841451,"POLYGON Z ((-90.31795 42.19391 0.00000, -90.22..."
3229,31,131,00835887,0500000US31131,31131,Otoe,06,1594475777,8775373,"POLYGON Z ((-96.46384 40.69696 0.00000, -96.46..."
3230,33,011,00873179,0500000US33011,33011,Hillsborough,06,2269220216,41604851,"POLYGON Z ((-72.06124 42.96584 0.00000, -72.05..."
3231,39,027,01074026,0500000US39027,39027,Clinton,06,1058488036,9350863,"POLYGON Z ((-84.00373 39.28847 0.00000, -84.00..."


In [10]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON Z ((-86.92120 32.65754 0.00000, -86.92...
1       POLYGON Z ((-85.74803 31.61918 0.00000, -85.74...
2       POLYGON Z ((-88.47323 31.89386 0.00000, -88.46...
3       POLYGON Z ((-88.13925 34.58779 0.00000, -88.13...
4       POLYGON Z ((-87.47308 32.30761 0.00000, -87.42...
                              ...                        
3228    POLYGON Z ((-90.31795 42.19391 0.00000, -90.22...
3229    POLYGON Z ((-96.46384 40.69696 0.00000, -96.46...
3230    POLYGON Z ((-72.06124 42.96584 0.00000, -72.05...
3231    POLYGON Z ((-84.00373 39.28847 0.00000, -84.00...
3232    POLYGON Z ((-83.26448 39.54067 0.00000, -83.26...
Name: geometry, Length: 3233, dtype: geometry

In [11]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0   -86.921196  32.657542
     1   -86.920352  32.658563
     2   -86.920411  32.660077
     3   -86.917595  32.664169
     4   -86.914610  32.664355
...             ...        ...
3232 120 -83.155472  39.511776
     121 -83.165611  39.512190
     122 -83.224839  39.514579
     123 -83.266737  39.516249
     124 -83.264481  39.540672

[1048659 rows x 2 columns]

In [12]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [13]:
%%time

## include all counties

df_list=[]

for i in range(0,3233): ## A total of 3232 counties. index in shape file starts from 0 and ends in 3232
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 5min 54s
Wall time: 6min


In [14]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,67.539761,0.009483
1,69.492739,0.009832
2,66.079332,0.009463
3,70.318496,0.008871
4,66.74961,0.009436
...,...,...
3228,71.744849,0.006263
3229,63.40987,0.006667
3230,68.010916,0.005648
3231,69.141633,0.00687


In [15]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

STATEFP       object
COUNTYFP      object
COUNTYNS      object
AFFGEOID      object
GEOID         object
NAME          object
LSAD          object
ALAND          int64
AWATER         int64
geometry    geometry
fips           int32
dtype: object

In [16]:
county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','AFFGEOID','GEOID','LSAD','ALAND','AWATER'], axis=1)
county_gdf

,NAME,geometry,fips
0,Autauga,"POLYGON Z ((-86.92120 32.65754 0.00000, -86.92...",1001
1,Barbour,"POLYGON Z ((-85.74803 31.61918 0.00000, -85.74...",1005
2,Choctaw,"POLYGON Z ((-88.47323 31.89386 0.00000, -88.46...",1023
3,Colbert,"POLYGON Z ((-88.13925 34.58779 0.00000, -88.13...",1033
4,Dallas,"POLYGON Z ((-87.47308 32.30761 0.00000, -87.42...",1047
...,...,...,...
3228,Carroll,"POLYGON Z ((-90.31795 42.19391 0.00000, -90.22...",17015
3229,Otoe,"POLYGON Z ((-96.46384 40.69696 0.00000, -96.46...",31131
3230,Hillsborough,"POLYGON Z ((-72.06124 42.96584 0.00000, -72.05...",33011
3231,Clinton,"POLYGON Z ((-84.00373 39.28847 0.00000, -84.00...",39027


In [17]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q
0,Autauga,"POLYGON Z ((-86.92120 32.65754 0.00000, -86.92...",1001,67.539761,0.009483
1,Barbour,"POLYGON Z ((-85.74803 31.61918 0.00000, -85.74...",1005,69.492739,0.009832
2,Choctaw,"POLYGON Z ((-88.47323 31.89386 0.00000, -88.46...",1023,66.079332,0.009463
3,Colbert,"POLYGON Z ((-88.13925 34.58779 0.00000, -88.13...",1033,70.318496,0.008871
4,Dallas,"POLYGON Z ((-87.47308 32.30761 0.00000, -87.42...",1047,66.74961,0.009436
...,...,...,...,...,...
3228,Carroll,"POLYGON Z ((-90.31795 42.19391 0.00000, -90.22...",17015,71.744849,0.006263
3229,Otoe,"POLYGON Z ((-96.46384 40.69696 0.00000, -96.46...",31131,63.40987,0.006667
3230,Hillsborough,"POLYGON Z ((-72.06124 42.96584 0.00000, -72.05...",33011,68.010916,0.005648
3231,Clinton,"POLYGON Z ((-84.00373 39.28847 0.00000, -84.00...",39027,69.141633,0.00687


In [18]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q
0,Autauga,"POLYGON Z ((-86.92120 32.65754 0.00000, -86.92...",1001,67.539761,0.009483
1,Barbour,"POLYGON Z ((-85.74803 31.61918 0.00000, -85.74...",1005,69.492739,0.009832
2,Choctaw,"POLYGON Z ((-88.47323 31.89386 0.00000, -88.46...",1023,66.079332,0.009463
3,Colbert,"POLYGON Z ((-88.13925 34.58779 0.00000, -88.13...",1033,70.318496,0.008871
4,Dallas,"POLYGON Z ((-87.47308 32.30761 0.00000, -87.42...",1047,66.74961,0.009436
...,...,...,...,...,...
3228,Carroll,"POLYGON Z ((-90.31795 42.19391 0.00000, -90.22...",17015,71.744849,0.006263
3229,Otoe,"POLYGON Z ((-96.46384 40.69696 0.00000, -96.46...",31131,63.40987,0.006667
3230,Hillsborough,"POLYGON Z ((-72.06124 42.96584 0.00000, -72.05...",33011,68.010916,0.005648
3231,Clinton,"POLYGON Z ((-84.00373 39.28847 0.00000, -84.00...",39027,69.141633,0.00687


In [19]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [20]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q
0,Autauga,1001,67.539760870467120,0.009483201614480
1,Barbour,1005,69.492738616728388,0.009832026798470
2,Choctaw,1023,66.079331633332629,0.009463291709042
3,Colbert,1033,70.318496224440636,0.008870773550088
4,Dallas,1047,66.749610027197207,0.009436053683618
...,...,...,...,...
3228,Carroll,17015,71.744849487852349,0.006263316081596
3229,Otoe,31131,63.409869834708168,0.006666900919232
3230,Hillsborough,33011,68.010915583737841,0.005648135091245
3231,Clinton,39027,69.141633315985104,0.006870310577491


In [21]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q
0,Autauga,1001,67.539760870467120,0.009483201614480
839,Baldwin,1003,72.833387916586048,0.011135148691435
1,Barbour,1005,69.492738616728388,0.009832026798470
1404,Bibb,1007,68.538625786753229,0.009360398180754
840,Blount,1009,69.480237443350461,0.009054339684136
...,...,...,...,...
1826,Sweetwater,56037,54.833520508324973,0.003817623833100
348,Teton,56039,69.393536796250558,0.003938033138221
1837,Uinta,56041,56.195224045155207,0.003776768793430
3170,Washakie,56043,55.596068547475276,0.004058921391191


In [22]:
county_var.to_pickle('era_2014.pkl')